In [1]:
import yaml

yaml_file_path = 'test.yaml'

with open(yaml_file_path, "r") as f:
    data = yaml.safe_load(f)

print(data)
print(type(data))

{'name': 'rootcode', 'age': 20, 'city': 'Dhaka', 'country': 'Bangladesh'}
<class 'dict'>


In [2]:
data['name']

'rootcode'

# Configbox

In [3]:
from box import ConfigBox

d = ConfigBox(data)

In [4]:
d

ConfigBox({'name': 'rootcode', 'age': 20, 'city': 'Dhaka', 'country': 'Bangladesh'})

In [5]:
d.name

'rootcode'

In [6]:
d.age

20

# ensure_annotations

In [7]:
def get_product(x:int, y:int):
    return x*y

In [9]:
get_product(x=2, y=4)

8

In [10]:
get_product(x=2, y="4")

'44'

In [14]:
from ensure import ensure_annotations

In [15]:
@ensure_annotations
def get_product(x:int, y:int):
    return x*y

In [16]:
get_product(x=2, y=4)

8

In [17]:
get_product(x=2, y="4")

EnsureError: Argument y of type <class 'str'> to <function get_product at 0x000001B6394FB6A0> does not match annotation type <class 'int'>

# S3 bucket data download 

In [ ]:
import boto3
import os
from dotenv import load_dotenv
from botocore.exceptions import ClientError

# Load environment variables from .env
load_dotenv()

def download_from_s3(bucket_name, object_key, download_path, 
                     aws_access_key_id, aws_secret_access_key, region_name="us-east-1"):
    # Initialize the S3 client with explicit credentials
    s3 = boto3.client(
        "s3",
        region_name=region_name,
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key
    )

    try:
        print(f"Downloading {object_key} from bucket {bucket_name} to {download_path}...")
        # Must include RequestPayer for Requester Pays buckets
        response = s3.get_object(
            Bucket=bucket_name,
            Key=object_key,
            RequestPayer="requester"
        )
        with open(download_path, "wb") as f:
            for chunk in response["Body"].iter_chunks(chunk_size=8*1024*1024):
                f.write(chunk)

        print("Download complete.")

    except ClientError as e:
        code = e.response["Error"]["Code"]
        print(f"Error ({code}): {e.response['Error']['Message']}")
        if code in ["403", "AccessDenied"]:
            print("→ Check IAM permissions and ensure RequestPayer='requester' is allowed in the bucket policy.")
        elif code == "PermanentRedirect":
            print("→ Wrong region: use get_bucket_location() to discover the correct region.")

# Example usage (⚠️ replace with your real keys — don’t commit them)
bucket_name = "chicken-data-2025"
object_key = "data.zip"
download_path = "data.zip"

download_from_s3(
    bucket_name,
    object_key,
    download_path,
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name="us-east-1"
)

Download complete.
